# Binary logistic regression model

In [1]:
import tensorflow as tf

/home/pablo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Create placeholders for inputs and labels

In [2]:
# Inputs
x = tf.placeholder(dtype=tf.float32, name="inputs")
# Labels
y = tf.placeholder(dtype=tf.float32, name="labels")

### Create Variable tensors for weights and bias
This tensors are initialized with random values

In [3]:
w = tf.Variable(tf.random_normal([2,1]), dtype=tf.float32, name="weight")
b = tf.Variable(tf.random_normal([1]), dtype=tf.float32, name="bias")

def linear_layer(x, w, b):
    
    return tf.matmul(x,w) + b

init = tf.global_variables_initializer()

### Building the model

In [4]:
output = tf.sigmoid(linear_layer(x, w, b))

### Cost and Optimizer

In [5]:
l = 1
with tf.name_scope("Cost_and_Optimizer"):
    cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=output,name="cost")
    optimizer = tf.train.GradientDescentOptimizer(l)
    train = optimizer.minimize(cost)

In [6]:
x_train = []
y_train = []

import csv

with open("./data/example_1.csv", 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        x_train.append([float(row['x1']),float(row['x2'])])
        y_train.append(float(row['y']))

In [7]:
tf.summary.scalar("Cost", cost)
tf.summary.histogram("weights", w)
tf.summary.histogram("biases", b)
epoch = 5000


with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./summaries/2")
    writer.add_graph(sess.graph)
    sess.run(init)
    for i in range(epoch):
        sess.run(train, feed_dict={x: x_train[i], y: y_train[i]})

        if i%40 == 0:
            partial_w, partial_b, partial_cost = sess.run([w, b, cost], {x: x_train, y: y_train})
            print("Current w: %s Current b: %s Current cost: %s"%(partial_w, partial_b, partial_cost))
            
            s = sess.run(merged_summary, {x: x_train, y: y_train})
            writer.add_summary(s, i)
        
    final_w, final_b, final_cost = sess.run([w, b, cost], {x: x_train, y: y_train})
    

InvalidArgumentError: In[0] is not a matrix
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_inputs_0_0, weight/read)]]

Caused by op 'MatMul', defined at:
  File "/home/pablo/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/pablo/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/pablo/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever

### Final model

In [ ]:
print("Final function: Y = %s * X + %s"%(final_w, final_b))
print("Final cost: %s"%final_cost)

### TensorBoard - Graph visualization
![title](graph.png)